In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(31, 'documents')
(10, 'classes', [u'appointment', u'goodbye', u'greeting', u'hours', u'mopeds', u'opentoday', u'payments', u'rental', u'thanks', u'today'])
(57, 'unique stemmed words', [u"'d", u"'s", u'a', u'acceiv', u'an', u'anyon', u'apooint', u'appoint', u'appt', u'ar', u'book', u'bye', u'can', u'card', u'cash', u'credit', u'day', u'do', u'doct', u'doe', u'good', u'goodby', u'hav', u'hello', u'help', u'hi', u'hour', u'how', u'i', u'is', u'ishit', u'kind', u'lat', u'lik', u'mak', u'mastercard', u'mop', u'nee', u'of', u'on', u'op', u'rent', u'see', u'tak', u'thank', u'that', u'ther', u'thi', u'to', u'today', u'we', u'what', u'when', u'which', u'work', u'yo', u'you'])


In [4]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
del tf.get_collection_ref(tf.GraphKeys.TRAIN_OPS)[:]
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.01146 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.01146 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.01199 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.01199 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/Users/pooja/tensorflow/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [7]:
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /Users/pooja/tensorflow/model.tflearn


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("is your shop open today?", words)
print (p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]


In [11]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    print(random.choice(i['responses']))
                    return random.choice(i['responses'])

            results.pop(0)

In [12]:
classify('is your shop open today?')


[(u'opentoday', 0.69702536), (u'today', 0.29097936)]

In [13]:
response('is your shop open today?')

Our hours are 9am-9pm every day


u'Our hours are 9am-9pm every day'

In [14]:
response('do you take cash?')


We accept most major credit cards


u'We accept VISA, Mastercard and AMEX'

In [15]:
response('Goodbye, see you later')

See you later, thanks for visiting


u'Have a nice day'

In [16]:
response('book appointment')

What day?


u'What day?'

In [17]:
response('is your shop open on saturday ?')


We're open every day 9am-9pm


u"We're open every day 9am-9pm"